In [1]:
!nvidia-smi

Sun Sep 13 06:27:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Mounted at /content/gdrive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
from sklearn.model_selection import train_test_split
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.image import resize
from tensorflow import keras
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import string

path = '/content/gdrive/My Drive/Dacon/ComputerVision/'
path_train = path + 'train.csv'
path_test = path + 'test.csv'
path_submission = path + 'submission.csv'

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
## Global Variables ##
ImageGen_coeff = 10
N_dim = 224
epochs_num = 80
verbose = 2
batch_size = 32
train_val_ratio = 0.9
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

In [ ]:
def prepare_datasets():
    '''
    ImageGen_coeff (Global Variable) : number of ImageDataGenerator generation per image
    N_dim (Global Variable) : we'll change the size of the image from (28,28) to (N_dim,N_dim)

    Since we don't have enough RAM for the resized image of test_pixel, we just return non-resized, non-standardized image.
    When we generate each batchs by making use of the tf.utils.Sequential class, we'll resize and standardize test_pixel.
    '''
    train_raw = pd.read_csv(path_train)

    letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))
    letter_hash_func = lambda letter : letter_hash[letter]

    ### only for validation 
    point_to = int(len(train_raw) * train_val_ratio)
    train_raw = train_raw.sample(frac=1)
    validation_raw = train_raw.iloc[point_to:,:]
    train_raw = train_raw.iloc[:point_to,:]
    
    with tf.device('/device:GPU:0'):
        validation_pixel = resize(validation_raw.iloc[:,3:].values.astype('int32').reshape(-1,28,28,1), [N_dim,N_dim], method = tf.image.ResizeMethod.BICUBIC).numpy()
    validation_pixel = validation_pixel / 255.0
    validation_fix = validation_raw.iloc[:,1:3].values
    datagenerator = ImageDataGenerator(rotation_range=10, zoom_range=0.10, width_shift_range=0.1,height_shift_range=0.1)
    gen = datagenerator.flow(validation_pixel, validation_fix, shuffle=False, batch_size=32)
    pixel, fixed, batch_index, limit = [], [], 0, int((len(validation_pixel) / 32) * ImageGen_coeff)
    while batch_index <= limit:
        try:
            data = gen.next()
            pixel += list(data[0])
            fixed += list(data[1])
            batch_index += 1
        except:
            print("ImageGeneratorError")
            break

    validation_pixel = np.asarray(pixel)
    validation_label = np.asarray(list(map(letter_hash_func, np.asarray(fixed)[:,1])))
    validation_answer = to_categorical(np.asarray(fixed)[:,0], 10)
    ###

    pix = train_raw.iloc[:,3:].values.astype('int32').reshape(-1,28,28,1)
    with tf.device('/device:GPU:0'):
        pix = resize(pix, [N_dim,N_dim], method = tf.image.ResizeMethod.BICUBIC).numpy()
    pix = pix / 255.0
    fix = train_raw.iloc[:,1:3].values
    datagenerator = ImageDataGenerator(rotation_range=10, zoom_range=0.10, width_shift_range=0.1,height_shift_range=0.1)
    gen = datagenerator.flow(pix, fix, shuffle=False, batch_size=32)
    pixel, fixed, batch_index, limit = [], [], 0, int((len(pix) / 32) * ImageGen_coeff)
    while batch_index <= limit:
        try:
            data = gen.next()
            pixel += list(data[0])
            fixed += list(data[1])
            batch_index += 1
        except:
            print("ImageGeneratorError")
            break

    train_pixel = np.asarray(pixel)
    train_label = np.asarray(list(map(letter_hash_func, np.asarray(fixed)[:,1])))
    train_answer = to_categorical(np.asarray(fixed)[:,0], 10)

    return train_pixel, train_label, train_answer, validation_pixel, validation_label, validation_answer

def prepare_testset():
    letter_hash = dict(zip(string.ascii_uppercase, [[1 if i == j else 0 for j in range(26)] for i in range(26)]))
    letter_hash_func = lambda letter : letter_hash[letter]

    test_raw = pd.read_csv(path_test)
    test_pixel = test_raw.iloc[:,2:].values.astype('int32').reshape(-1,28,28,1)
    test_label = np.asarray(list(map(letter_hash_func, test_raw.iloc[:,1].values))).astype('float32')
    return test_pixel, test_label

test_pixel, test_label = prepare_testset()
print(test_pixel.shape, test_label.shape)

(20480, 28, 28, 1) (20480, 26)


In [ ]:
def train_test_model(i):
    tf.keras.backend.clear_session()
    '''
    I shuffled dataset every time since i wanted to get enoughly distributed models.
    '''
    train_pixel, train_label, train_answer, valid_pixel, valid_label, valid_answer = prepare_datasets()
    callbacks1 = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True), ReduceLROnPlateau(monitor = 'loss', patience = 5)]
    ModelName = 'InceptionResnetV2_bicubic224_val'
    print(ModelName + '  ' + str(i))
    with tf.device('/device:GPU:0'):
        cnn = tf.keras.applications.InceptionResNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(cnn.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([cnn.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        history = concat_model.fit([train_pixel, train_label], train_answer, validation_data=([valid_pixel, valid_label], valid_answer), epochs=epochs_num, verbose=verbose, callbacks = callbacks1)
    print(
        f"Train accuracy={max(history.history['accuracy']):.5f}, " +
        f"Validation accuracy={max(history.history['val_accuracy']):.5f}"
    )
    with tf.device('/device:GPU:0'):
        concat_model.fit([valid_pixel, valid_label], valid_answer, epochs=5, verbose=verbose)
    concat_model.save(path + ModelName + '_' + str(i) + '.h5')

for i in range(5):
    train_test_model(i)

InceptionResnetV2_bicubic224_val  0
Epoch 1/80
573/573 - 126s - loss: 0.7789 - accuracy: 0.7397 - val_loss: 2.3420 - val_accuracy: 0.3965
Epoch 2/80
573/573 - 122s - loss: 0.2694 - accuracy: 0.9127 - val_loss: 0.7520 - val_accuracy: 0.7800
Epoch 3/80
573/573 - 122s - loss: 0.1549 - accuracy: 0.9483 - val_loss: 0.6956 - val_accuracy: 0.7805
Epoch 4/80
573/573 - 122s - loss: 0.1128 - accuracy: 0.9630 - val_loss: 0.3771 - val_accuracy: 0.8900
Epoch 5/80
573/573 - 122s - loss: 0.0930 - accuracy: 0.9698 - val_loss: 0.4955 - val_accuracy: 0.8806
Epoch 6/80
573/573 - 122s - loss: 0.0737 - accuracy: 0.9761 - val_loss: 0.6902 - val_accuracy: 0.8434
Epoch 7/80
573/573 - 122s - loss: 0.0515 - accuracy: 0.9824 - val_loss: 0.9045 - val_accuracy: 0.8140
Epoch 8/80
573/573 - 122s - loss: 0.0575 - accuracy: 0.9821 - val_loss: 0.6825 - val_accuracy: 0.8434
Epoch 9/80
573/573 - 122s - loss: 0.0428 - accuracy: 0.9862 - val_loss: 0.7333 - val_accuracy: 0.8318
Epoch 10/80
573/573 - 122s - loss: 0.0354 - ac

In [ ]:
def train_test_model(i):
    tf.keras.backend.clear_session()
    '''
    I shuffled dataset every time since i wanted to get enoughly distributed models.
    '''
    train_pixel, train_label, train_answer, valid_pixel, valid_label, valid_answer = prepare_datasets()
    callbacks1 = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 20, restore_best_weights = True), ReduceLROnPlateau(monitor = 'loss', patience = 5)]
    ModelName = 'EfficientNetB5_bicubic224_val'
    print(ModelName + '  ' + str(i))
    with tf.device('/device:GPU:0'):
        cnn = tf.keras.applications.EfficientNetB5(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(cnn.output)
        cnn_out = layers.Dense(128, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(26,))
        dense_mid = layers.Dense(52, activation = 'relu')(dense_input)
        dense_output = layers.Dense(52, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([cnn.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        history = concat_model.fit([train_pixel, train_label], train_answer, validation_data=([valid_pixel, valid_label], valid_answer), epochs=epochs_num, verbose=verbose, callbacks = callbacks1)
    print(
        f"Train accuracy={max(history.history['accuracy']):.5f}, " +
        f"Validation accuracy={max(history.history['val_accuracy']):.5f}"
    )
    with tf.device('/device:GPU:0'):
        concat_model.fit([valid_pixel, valid_label], valid_answer, epochs=5, verbose=verbose)
    concat_model.save(path + ModelName + '_' + str(i) + '.h5')

for i in range(5):
    train_test_model(i)

def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
            pixel = pixel / 255.0

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['InceptionResnetV2_bicubic224_val_' + str(i) for i in range(5)] + ['EfficientNetB5_bicubic224_val_' + str(i) for i in range(5)]
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)
print('Done!')

EfficientNetB5_bicubic224_val  0
Epoch 1/50
573/573 - 258s - loss: 1.6866 - accuracy: 0.3872 - val_loss: 5.6478 - val_accuracy: 0.0786
Epoch 2/50
573/573 - 252s - loss: 0.6351 - accuracy: 0.7963 - val_loss: 0.8901 - val_accuracy: 0.7575
Epoch 3/50
573/573 - 253s - loss: 0.3701 - accuracy: 0.8801 - val_loss: 0.8070 - val_accuracy: 0.7753
Epoch 4/50
573/573 - 254s - loss: 0.2073 - accuracy: 0.9316 - val_loss: 0.5552 - val_accuracy: 0.8481
Epoch 5/50
573/573 - 252s - loss: 0.1553 - accuracy: 0.9497 - val_loss: 0.6600 - val_accuracy: 0.8371
Epoch 6/50
573/573 - 253s - loss: 0.1168 - accuracy: 0.9622 - val_loss: 0.7120 - val_accuracy: 0.8130
Epoch 7/50
573/573 - 253s - loss: 0.1034 - accuracy: 0.9669 - val_loss: 1.0728 - val_accuracy: 0.7664
Epoch 8/50
573/573 - 252s - loss: 0.0787 - accuracy: 0.9746 - val_loss: 1.3748 - val_accuracy: 0.7135
Epoch 9/50
573/573 - 253s - loss: 0.0848 - accuracy: 0.9732 - val_loss: 0.5624 - val_accuracy: 0.8565
Epoch 10/50
573/573 - 252s - loss: 0.0718 - accur

Progress:[==============================] 100.0%


Done!


In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
            pixel = pixel / 255.0

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['InceptionResnetV2_bicubic224_val_' + str(i) for i in range(5)]
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)
print('Done!')

Progress:[==============================] 100.0%


Done!


가장 좋은넘인 effi-4 만 써보자.
---------

In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
            pixel = pixel / 255.0

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['EfficientNetB5_bicubic224_val_' + str(i) for i in range(4,5)]
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)
print('Done!')

Progress:[==============================] 100.0%


Done!


93.1...ensemble 7
--------

In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
            pixel = pixel / 255.0

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['InceptionResnetV2_bicubic224_val_' + str(i) for i in [0,2,3]] + ['EfficientNetB5_bicubic224_val_' + str(i) for i in [1,2,3,4]]
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)
print('Done!')

Progress:[==============================] 100.0%


Done!


ensemble 6
---------

In [ ]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
            pixel = pixel / 255.0

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['InceptionResnetV2_bicubic224_val_' + str(i) for i in [0,2]] + ['EfficientNetB5_bicubic224_val_' + str(i) for i in [1,2,3,4]]
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)
print('Done!')

Progress:[==============================] 100.0%


Done!


k-mean

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import math
from sklearn.model_selection import train_test_split
from sklearn import cluster
from skimage import feature
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow.image import resize
from tensorflow import keras
from tensorflow.keras.utils import to_categorical, Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import string

path = '/content/gdrive/My Drive/Dacon/ComputerVision/'
path_train = path + 'train.csv'
path_test = path + 'test.csv'
path_submission = path + 'submission.csv'

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
## Global Variables ##
ImageGen_coeff = 10
N_dim = 224
epochs_num = 80
verbose = 2
batch_size = 32
train_val_ratio = 0.9
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

In [5]:
def image_updown_clustering(df1, df2):
    for alphabet_large in string.ascii_uppercase:
        alphabet_small = alphabet_large.lower()
        img_converted = []
        images_alphabet1 = df1[df1.iloc[:,2] == alphabet_large].iloc[:,3:]
        images_alphabet2 = df2[df2.iloc[:,1] == alphabet_large].iloc[:,2:]
        real_idxs1 = list(images_alphabet1.index)
        real_idxs2 = list(images_alphabet2.index)
        pixels = np.concatenate((images_alphabet1.values.reshape(-1,28,28), images_alphabet2.values.reshape(-1,28,28)), axis=0) 
        for pixel in pixels:
            img_converted.append(feature.hog(pixel))
        k_means = cluster.KMeans(n_clusters = 2)
        k_means.fit(np.asarray(img_converted))

        for idx, new_label in zip(real_idxs1, k_means.labels_[:len(real_idxs1)]):
            if new_label == 0:
                df1.loc[idx,'letter'] = alphabet_small
        for idx, new_label in zip(real_idxs2, k_means.labels_[len(real_idxs1):]):
            if new_label == 0:
                df2.loc[idx,'letter'] = alphabet_small
        
    return df1, df2

train_raw, test_raw = pd.read_csv(path_train), pd.read_csv(path_test)
train_raw, test_raw = image_updown_clustering(train_raw, test_raw)

def prepare_datasets(train_raw):
    '''
    ImageGen_coeff (Global Variable) : number of ImageDataGenerator generation per image
    N_dim (Global Variable) : we'll change the size of the image from (28,28) to (N_dim,N_dim)

    Since we don't have enough RAM for the resized image of test_pixel, we just return non-resized, non-standardized image.
    When we generate each batchs by making use of the tf.utils.Sequential class, we'll resize and standardize test_pixel.
    '''

    letter_hash = dict(zip(string.ascii_uppercase + string.ascii_lowercase, [[1 if i == j else 0 for j in range(52)] for i in range(52)]))
    letter_hash_func = lambda letter : letter_hash[letter]

    ### only for validation 
    point_to = int(len(train_raw) * train_val_ratio)
    train_raw = train_raw.sample(frac=1)
    validation_raw = train_raw.iloc[point_to:,:]
    train_raw = train_raw.iloc[:point_to,:]
    
    with tf.device('/device:GPU:0'):
        validation_pixel = resize(validation_raw.iloc[:,3:].values.astype('int32').reshape(-1,28,28,1), [N_dim,N_dim], method = tf.image.ResizeMethod.BICUBIC).numpy()
    validation_pixel = validation_pixel / 255.0
    validation_fix = validation_raw.iloc[:,1:3].values
    datagenerator = ImageDataGenerator(rotation_range=10, zoom_range=0.10, width_shift_range=0.1,height_shift_range=0.1)
    gen = datagenerator.flow(validation_pixel, validation_fix, shuffle=False, batch_size=32)
    pixel, fixed, batch_index, limit = [], [], 0, int((len(validation_pixel) / 32) * ImageGen_coeff)
    while batch_index <= limit:
        try:
            data = gen.next()
            pixel += list(data[0])
            fixed += list(data[1])
            batch_index += 1
        except:
            print("ImageGeneratorError")
            break

    validation_pixel = np.asarray(pixel)
    validation_label = np.asarray(list(map(letter_hash_func, np.asarray(fixed)[:,1])))
    validation_answer = to_categorical(np.asarray(fixed)[:,0], 10)
    ###

    pix = train_raw.iloc[:,3:].values.astype('int32').reshape(-1,28,28,1)
    with tf.device('/device:GPU:0'):
        pix = resize(pix, [N_dim,N_dim], method = tf.image.ResizeMethod.BICUBIC).numpy()
    pix = pix / 255.0
    fix = train_raw.iloc[:,1:3].values
    datagenerator = ImageDataGenerator(rotation_range=10, zoom_range=0.10, width_shift_range=0.1,height_shift_range=0.1)
    gen = datagenerator.flow(pix, fix, shuffle=False, batch_size=32)
    pixel, fixed, batch_index, limit = [], [], 0, int((len(pix) / 32) * ImageGen_coeff)
    while batch_index <= limit:
        try:
            data = gen.next()
            pixel += list(data[0])
            fixed += list(data[1])
            batch_index += 1
        except:
            print("ImageGeneratorError")
            break

    train_pixel = np.asarray(pixel)
    train_label = np.asarray(list(map(letter_hash_func, np.asarray(fixed)[:,1])))
    train_answer = to_categorical(np.asarray(fixed)[:,0], 10)

    return train_pixel, train_label, train_answer, validation_pixel, validation_label, validation_answer

def prepare_testset(test_raw):
    letter_hash = dict(zip(string.ascii_uppercase + string.ascii_lowercase, [[1 if i == j else 0 for j in range(52)] for i in range(52)]))
    letter_hash_func = lambda letter : letter_hash[letter]

    test_pixel = test_raw.iloc[:,2:].values.astype('int32').reshape(-1,28,28,1)
    test_label = np.asarray(list(map(letter_hash_func, test_raw.iloc[:,1].values))).astype('float32')
    return test_pixel, test_label

test_pixel, test_label = prepare_testset(test_raw)
print(test_pixel.shape, test_label.shape)

(20480, 28, 28, 1) (20480, 52)


In [ ]:
def train_test_model(i):
    tf.keras.backend.clear_session()
    '''
    I shuffled dataset every time since i wanted to get enoughly distributed models.
    '''
    train_pixel, train_label, train_answer, valid_pixel, valid_label, valid_answer = prepare_datasets(train_raw)
    callbacks1 = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 15, restore_best_weights = True), ReduceLROnPlateau(monitor = 'loss', patience = 5)]
    ModelName = 'InceptionResnetV2_bicubic224_val_kmean'
    print(ModelName + '  ' + str(i))
    with tf.device('/device:GPU:0'):
        cnn = tf.keras.applications.InceptionResNetV2(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(cnn.output)
        cnn_out = layers.Dense(256, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(52,))
        dense_mid = layers.Dense(104, activation = 'relu')(dense_input)
        dense_output = layers.Dense(104, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([cnn.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        history = concat_model.fit([train_pixel, train_label], train_answer, validation_data=([valid_pixel, valid_label], valid_answer), epochs=epochs_num, verbose=verbose, callbacks = callbacks1)
    print(
        f"Train accuracy={max(history.history['accuracy']):.5f}, " +
        f"Validation accuracy={max(history.history['val_accuracy']):.5f}"
    )
    with tf.device('/device:GPU:0'):
        concat_model.fit([valid_pixel, valid_label], valid_answer, epochs=5, verbose=verbose)
    concat_model.save(path + ModelName + '_' + str(i) + '.h5')

for i in range(5):
    train_test_model(i)

def train_test_model(i):
    tf.keras.backend.clear_session()
    '''
    I shuffled dataset every time since i wanted to get enoughly distributed models.
    '''
    train_pixel, train_label, train_answer, valid_pixel, valid_label, valid_answer = prepare_datasets(train_raw)
    callbacks1 = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 20, restore_best_weights = True), ReduceLROnPlateau(monitor = 'loss', patience = 5)]
    ModelName = 'EfficientNetB5_bicubic224_val_kmean'
    print(ModelName + '  ' + str(i))
    with tf.device('/device:GPU:0'):
        cnn = tf.keras.applications.EfficientNetB5(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(cnn.output)
        cnn_out = layers.Dense(256, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(52,))
        dense_mid = layers.Dense(104, activation = 'relu')(dense_input)
        dense_output = layers.Dense(104, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([cnn.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        history = concat_model.fit([train_pixel, train_label], train_answer, validation_data=([valid_pixel, valid_label], valid_answer), epochs=epochs_num, verbose=verbose, callbacks = callbacks1)
    print(
        f"Train accuracy={max(history.history['accuracy']):.5f}, " +
        f"Validation accuracy={max(history.history['val_accuracy']):.5f}"
    )
    with tf.device('/device:GPU:0'):
        concat_model.fit([valid_pixel, valid_label], valid_answer, epochs=5, verbose=verbose)
    concat_model.save(path + ModelName + '_' + str(i) + '.h5')

for i in range(5):
    train_test_model(i)

InceptionResnetV2_bicubic224_val_kmean  0
Epoch 1/80
573/573 - 124s - loss: 1.4260 - accuracy: 0.4862 - val_loss: 3.6134 - val_accuracy: 0.2708
Epoch 2/80
573/573 - 121s - loss: 0.3741 - accuracy: 0.8763 - val_loss: 0.5375 - val_accuracy: 0.8539
Epoch 3/80
573/573 - 120s - loss: 0.1860 - accuracy: 0.9366 - val_loss: 4.8396 - val_accuracy: 0.4133
Epoch 4/80
573/573 - 121s - loss: 0.1153 - accuracy: 0.9614 - val_loss: 0.6092 - val_accuracy: 0.8523
Epoch 5/80
573/573 - 121s - loss: 0.0890 - accuracy: 0.9709 - val_loss: 0.8589 - val_accuracy: 0.8324
Epoch 6/80
573/573 - 121s - loss: 0.0763 - accuracy: 0.9757 - val_loss: 0.6781 - val_accuracy: 0.8439
Epoch 7/80
573/573 - 121s - loss: 0.0685 - accuracy: 0.9797 - val_loss: 0.5578 - val_accuracy: 0.8654
Epoch 8/80
573/573 - 121s - loss: 0.0540 - accuracy: 0.9832 - val_loss: 0.8265 - val_accuracy: 0.8465
Epoch 9/80
573/573 - 120s - loss: 0.0454 - accuracy: 0.9860 - val_loss: 1.0220 - val_accuracy: 0.8172
Epoch 10/80
573/573 - 120s - loss: 0.049

In [ ]:
def train_test_model(i):
    tf.keras.backend.clear_session()
    '''
    I shuffled dataset every time since i wanted to get enoughly distributed models.
    '''
    train_pixel, train_label, train_answer, valid_pixel, valid_label, valid_answer = prepare_datasets(train_raw)
    callbacks1 = [tf.keras.callbacks.EarlyStopping(monitor = 'val_accuracy', patience = 20, restore_best_weights = True), ReduceLROnPlateau(monitor = 'loss', patience = 5)]
    ModelName = 'EfficientNetB5_bicubic224_val_kmean'
    print(ModelName + '  ' + str(i))
    with tf.device('/device:GPU:0'):
        cnn = tf.keras.applications.EfficientNetB5(include_top=False, weights=None, input_tensor=None, input_shape=(N_dim,N_dim,1),pooling=None)
        cnn_mid = layers.GlobalAveragePooling2D()(cnn.output)
        cnn_out = layers.Dense(256, activation = 'relu')(cnn_mid)

        dense_input = layers.Input(shape=(52,))
        dense_mid = layers.Dense(104, activation = 'relu')(dense_input)
        dense_output = layers.Dense(104, activation = 'relu')(dense_mid)
        dense_model = tf.keras.Model(inputs=dense_input, outputs=dense_output)

        concatenated = layers.concatenate([cnn_out, dense_model.output])
        concatenated = layers.Dense(32, activation='relu')(concatenated)
        concat_output = layers.Dense(10, activation='softmax')(concatenated)
        concat_model = tf.keras.models.Model([cnn.input, dense_input], concat_output)
        concat_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
        history = concat_model.fit([train_pixel, train_label], train_answer, validation_data=([valid_pixel, valid_label], valid_answer), epochs=epochs_num, verbose=verbose, callbacks = callbacks1)
    print(
        f"Train accuracy={max(history.history['accuracy']):.5f}, " +
        f"Validation accuracy={max(history.history['val_accuracy']):.5f}"
    )
    with tf.device('/device:GPU:0'):
        concat_model.fit([valid_pixel, valid_label], valid_answer, epochs=5, verbose=verbose)
    concat_model.save(path + ModelName + '_' + str(i) + '.h5')

for i in range(3,5):
    train_test_model(i)

EfficientNetB5_bicubic224_val_kmean  3
Epoch 1/80
573/573 - 257s - loss: 1.6888 - accuracy: 0.3994 - val_loss: 6.0406 - val_accuracy: 0.0686
Epoch 2/80
573/573 - 254s - loss: 0.7882 - accuracy: 0.7467 - val_loss: 0.4887 - val_accuracy: 0.8497
Epoch 3/80
573/573 - 253s - loss: 0.3312 - accuracy: 0.8925 - val_loss: 0.5430 - val_accuracy: 0.8423
Epoch 4/80
573/573 - 253s - loss: 0.2228 - accuracy: 0.9275 - val_loss: 0.7127 - val_accuracy: 0.7905
Epoch 5/80
573/573 - 253s - loss: 0.1549 - accuracy: 0.9508 - val_loss: 0.4687 - val_accuracy: 0.8649
Epoch 6/80
573/573 - 253s - loss: 0.1347 - accuracy: 0.9580 - val_loss: 0.4665 - val_accuracy: 0.8675
Epoch 7/80
573/573 - 252s - loss: 0.1099 - accuracy: 0.9638 - val_loss: 0.4641 - val_accuracy: 0.8753
Epoch 8/80
573/573 - 253s - loss: 0.0867 - accuracy: 0.9712 - val_loss: 0.5696 - val_accuracy: 0.8795
Epoch 9/80
573/573 - 253s - loss: 0.0888 - accuracy: 0.9706 - val_loss: 0.5759 - val_accuracy: 0.8743
Epoch 10/80
573/573 - 253s - loss: 0.0704 -

In [9]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
            pixel = pixel / 255.0

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['InceptionResnetV2_bicubic224_val_kmean_' + str(i) for i in range(4)] + ['EfficientNetB5_bicubic224_val_kmean_' + str(i) for i in range(4)]
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)
print('Done!')

Progress:[==============================] 100.0%


Done!


In [11]:
def get_progressbar_str(progress):
    MAX_LEN = 30
    BAR_LEN = int(MAX_LEN * progress)
    return ('Progress:[' + '=' * BAR_LEN +
            ('>' if BAR_LEN < MAX_LEN else '') +
            ' ' * (MAX_LEN - BAR_LEN) +
            '] %.1f%%' % (progress * 100.))

class ComputerVision_Data_test():
    def __init__(self, pixel, label, batch_size):
        self.pixel = pixel
        self.label =  label
        self.batch_size = batch_size
        self.length = math.ceil(len(label) // batch_size)

    def __getitem__(self, idx):
        i = idx * self.batch_size
        if i + self.batch_size > len(self.label):
            pixel = self.pixel[i:]
            label = self.label[i:]
        else:
            pixel = self.pixel[i:i+self.batch_size]
            label = self.label[i:i+self.batch_size]
        with tf.device('/device:GPU:0'):
            pixel = resize(pixel, [224,224], method=tf.image.ResizeMethod.BICUBIC).numpy()
            pixel = pixel / 255.0

        return [pixel, label]

    def __len__(self):
        return self.length


test_dataset = ComputerVision_Data_test(test_pixel, test_label, batch_size = 1000)

batch_idxs = [i for i in range(0, len(test_dataset))]
ModelNames = ['InceptionResnetV2_bicubic224_val_kmean_' + str(i) for i in [2]] + ['EfficientNetB5_bicubic224_val_kmean_' + str(i) for i in [3]]
batch_size = test_dataset.batch_size
submission = pd.read_csv(path_submission)
submission_temp = pd.DataFrame(columns = list(range(10)),index = list(range(len(submission)))).fillna(0)
inc = 0
L = len(batch_idxs) * len(ModelNames)
for ModelName in ModelNames:
    tf.keras.backend.clear_session()
    model = tf.keras.models.load_model(path+ModelName+'.h5')
    for batch_idx in batch_idxs:
        target = test_dataset[batch_idx]
        real_idxs = [i for i in range(batch_size * batch_idx, batch_size * batch_idx + len(target[0])) ]
        with tf.device('/device:GPU:0'):
            predicted = model.predict(target)
        for idx_temp, real in enumerate(real_idxs):
            submission_temp.iloc[real,:] += predicted[idx_temp]
        inc += 1
        sys.stderr.write('\r\033[K' + get_progressbar_str(inc/L))
        sys.stderr.flush()
sys.stderr.write('\n')
sys.stderr.flush()
submission.iloc[:,1] = submission_temp.apply(np.argmax,axis=1).apply(int).values
submission.to_csv(path+'submission_ensemble_'+str(len(ModelNames))+'.csv',index = False)
print('Done!')

Progress:[==============================] 100.0%


Done!
